# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

Some explantations on abbrevations used:

#### SFT - Supervised Fine-Tuning
The process of further training a pre-trained language model on labeled datasets to adapt it to specific tasks or domains.

#### RLHF - Reinforcement Learning from Human Feedback
A technique that uses human feedback to guide and improve a model’s behavior through reinforcement learning.

#### DPO - Direct Preference Optimization
A method for training language models by directly optimizing for human preferences without requiring reward modeling.

#### TRL - Transformer Reinforcement Learning
A toolkit built on top of Hugging Face’s Transformers library that enables reinforcement learning-based training of transformer models.

#### RL - Reinforcement Learning
A machine learning paradigm where an agent learns optimal behavior through trial and error, receiving rewards or penalties based on its actions.

#### RM - Reward Modeling
A process in reinforcement learning where a model is trained to predict the reward signal used to guide further optimization.

#### PPO - Proximal Policy Optimization
A reinforcement learning algorithm that optimizes a policy in a stable and efficient manner, commonly used in training AI agents, including language models.


In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [18]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-CustomerSupportBot"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [19]:
# Let's test the base model before training
prompt = "I want to request a refund for my order"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order
I want to request a refund for my order



## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [10]:
# Load a sample dataset
from datasets import load_dataset

# Load dataset
ds = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", name="default")

# Convert dataset to chat template format while keeping it structured
def process_chat_template(example):
    return {
        "input_ids": tokenizer.apply_chat_template(
            [{"role": "user", "content": example["instruction"]},
             {"role": "assistant", "content": example["response"]}],
            tokenize=True
        )
    }

ds = ds.map(process_chat_template)

# dataset doesn't have a train/test split, let's do that with 10 percent
split_ds = ds["train"].train_test_split(test_size=0.1)



Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [11]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=split_ds["train"],
    tokenizer=tokenizer,
    eval_dataset=split_ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/home/nurbot/miniconda3/envs/lchain/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_58934/2044347143.py:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [12]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

/home/nurbot/miniconda3/envs/lchain/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
50,1.260200,1.225711
100,1.072300,1.072031
150,0.993100,1.003623
200,0.947700,0.955952
250,0.947400,0.925250
300,0.918100,0.899610
350,0.896900,0.881896
400,0.857600,0.862807
450,0.822300,0.851489
500,0.841700,0.839100


/home/nurbot/miniconda3/envs/lchain/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/nurbot/miniconda3/envs/lchain/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/nurbot/miniconda3/envs/lchain/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/nurbot/miniconda3/envs/lchain/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [13]:
trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

events.out.tfevents.1737119893.server.58934.0:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/tobrun-mapbox/SmolLM2-FT-CustomerSupportBot/commit/b442e072b3d112104dc70497290237142a194635', commit_message='End of training', commit_description='', oid='b442e072b3d112104dc70497290237142a194635', pr_url=None, pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [22]:
# Test the fine-tuned model
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Load the fine-tuned model and tokenizer
finetuned_model_name = "./SmolLM2-FT-CustomerSupportBot"
model = AutoModelForCausalLM.from_pretrained(finetuned_model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(finetuned_model_name)

# Test the fine-tuned model
prompt = "I want to ask for a refund of my order 50"

# Format prompt using chat template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tokenize input
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Generate response
outputs = model.generate(**inputs, max_new_tokens=100)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(response)

<|im_start|>user
I want to ask for a refund of my order 50<|im_end|>
<|im_start|>assistant
I'm sorry to hear that you're seeking a refund for your order. I understand that you're frustrated and would like to seek a refund. To assist you better, could you please provide me with more details about your order? This will help me understand the situation and provide you with the most accurate and efficient solution. Rest assured, I'm here to guide you through the refund process and ensure that you receive the assistance you need. Thank you for reaching out to us, and


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.